In [1]:
# First 6 weeks & rewards in 19 Audience only 

import pandas as pd
import datetime
import numpy as np
import os
import glob
import gc

def recursive_file_gen(root_folder):
    for root, dirs, files in os.walk(root_folder):
        for file in files:
            yield os.path.join(root, file)
            
os.getcwd()

# Note how to difine the new sign ups

'/home/jian/Projects/Big_Lots/Analysis/2019_Q3/Projection_est_in_Q3'

In [2]:
set_SOTF_number=pd.ExcelFile("./SOTF/Store List Report 20190910  09.10.19 417PM Dom edited BR .xlsx")
# set_SOTF_number.sheet_names
set_SOTF_number=set_SOTF_number.parse("SOTF pre 11-15-19 (462)",dtype=str)
set_SOTF_number=set_SOTF_number[(set_SOTF_number['SOTF']=="Yes") & (set_SOTF_number['Open Date']<="2019-09-07")]
set_SOTF_number=set(set_SOTF_number['Store'].unique().tolist())
len(set_SOTF_number)

446

In [3]:
new_sign_ups_after_2019Q3_end=pd.read_table("/home/jian/BigLots/New_Sing_Ups_2018_Fiscal_Year/All Rewards Members 2018-02-04 - 2019-05-04.zip",
                                             compression="zip",dtype=str,sep="|",usecols=['customer_id_hashed','sign_up_date'])
new_sign_ups_after_2019Q3_end=new_sign_ups_after_2019Q3_end[new_sign_ups_after_2019Q3_end['sign_up_date']>="2018-08-05"] # First of 2018Q3
master_file_gap=pd.read_table("/home/jian/BigLots/New_Sing_Ups_2018_Fiscal_Year/MediaStorm Rewards Master P4 2019 - no transaction info.zip",
                                             compression="zip",dtype=str,sep="|",usecols=['customer_id_hashed','sign_up_date'])
new_sign_ups_after_2019Q3_end=master_file_gap.append(new_sign_ups_after_2019Q3_end).drop_duplicates()
del master_file_gap
###

all_master_files_after_201906=list(recursive_file_gen("/home/jian/BigLots/2019_by_weeks/"))
all_master_files_after_201906=[x for x in all_master_files_after_201906 if ("aster" in x) & (".txt" in x)]
all_master_files_after_201906=[x for x in all_master_files_after_201906 if x.split("ts/2019_by_weeks/MediaStorm_")[1][:10]>"2019-06-01"]
df_master_all_later_after_201906=pd.DataFrame()
for file in all_master_files_after_201906:
    df=pd.read_table(file,dtype=str,sep="|",usecols=['customer_id_hashed','sign_up_date'])
    df_master_all_later_after_201906=df_master_all_later_after_201906.append(df)

new_sign_ups_after_2019Q3_end=new_sign_ups_after_2019Q3_end.append(df_master_all_later_after_201906).drop_duplicates()
del df_master_all_later_after_201906


gc.collect()
print("new_sign_ups_after_2019Q3_end.shape",new_sign_ups_after_2019Q3_end.shape)
print("new_sign_ups_after_2019Q3_end['customer_id_hashed'].nunique()",new_sign_ups_after_2019Q3_end['customer_id_hashed'].nunique())

new_sign_ups_after_2019Q3_end.shape (7623850, 2)
new_sign_ups_after_2019Q3_end['customer_id_hashed'].nunique() 7623841


In [4]:
dup_ids=new_sign_ups_after_2019Q3_end[new_sign_ups_after_2019Q3_end['customer_id_hashed'].duplicated()]['customer_id_hashed'].tolist()
df=new_sign_ups_after_2019Q3_end[new_sign_ups_after_2019Q3_end['customer_id_hashed'].isin(dup_ids)]
df.shape

(18, 2)

In [5]:
new_sign_ups_after_2019Q3_end=new_sign_ups_after_2019Q3_end.sort_values(['customer_id_hashed','sign_up_date'],ascending=[True,False]).drop_duplicates("customer_id_hashed")
set_new_signed_ids=set(new_sign_ups_after_2019Q3_end['customer_id_hashed'].tolist())
del new_sign_ups_after_2019Q3_end
gc.collect()

35

In [6]:
audience_list_2019Q3="/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q3/upload_files/"
audience_list_2019Q3=glob.glob(audience_list_2019Q3+"*.csv")
len(audience_list_2019Q3)


audience_list_2019Q3_C=[x for x in audience_list_2019Q3 if os.path.basename(x)[0]=="C"]
audience_list_2019Q3_T=[x for x in audience_list_2019Q3 if os.path.basename(x)[0]=="T"]
print(len(audience_list_2019Q3_C),len(audience_list_2019Q3_T))

set_C_2019Q3=set()
for file in audience_list_2019Q3_C:
    df=pd.read_csv(file,dtype=str,usecols=["customer_id_hashed"])
    set_C_2019Q3=set_C_2019Q3.union(set(df['customer_id_hashed'].tolist()))
print(len(set_C_2019Q3))


df_audience_2019Q3=pd.DataFrame()

for file in audience_list_2019Q3_T:
    df=pd.read_csv(file,dtype=str,usecols=['customer_id_hashed','segment'])
    df=df[~df['customer_id_hashed'].isin(set_C_2019Q3)]
    df_audience_2019Q3=df_audience_2019Q3.append(df)
df_audience_2019Q3=df_audience_2019Q3.drop_duplicates()

print(df_audience_2019Q3.shape," | ",df_audience_2019Q3['customer_id_hashed'].nunique())

25 25
542702
(21810554, 2)  |  21810554


In [7]:
# Rolling up removing the quadrant
df_audience_total_count_original=df_audience_2019Q3.groupby("segment")['customer_id_hashed'].nunique().to_frame().reset_index().rename(columns={"customer_id_hashed":"ids_total"})
df_audience_2019Q3['segment_2']=df_audience_2019Q3['segment'].str.replace("Quadrant I_","")
df_audience_2019Q3['segment_2']=df_audience_2019Q3['segment_2'].str.replace("Quadrant II_","")
df_audience_2019Q3['segment_2']=df_audience_2019Q3['segment_2'].str.replace("Quadrant III_","")
df_audience_2019Q3['segment_2']=df_audience_2019Q3['segment_2'].str.replace("Quadrant IV_","")
print(df_audience_2019Q3.shape)

df_audience_total_count=df_audience_2019Q3.groupby("segment_2")['customer_id_hashed'].nunique().to_frame().reset_index().rename(columns={"customer_id_hashed":"ids_total"})

del df_audience_2019Q3['segment']
df_audience_2019Q3=df_audience_2019Q3.rename(columns={"segment_2":"segment"})

print(df_audience_total_count.shape)
df_audience_total_count

(21810554, 3)
(7, 2)


,segment_2,ids_total
0,T_P_H_2019Q3_RFM,4937981
1,T_P_L_2019Q3_RFM,3391401
2,T_P_M_2019Q3_RFM,3515875
3,T_S_H_2019Q3_RFM,1694485
4,T_S_L_2019Q3_RFM,1422640
5,T_S_M_2019Q3_RFM,1361003
6,T_lapsed_18_48_PS_RFM,5487169


In [10]:
# Read from the deat session, the POS data is passed
print(datetime.datetime.now())

2019-09-19 12:36:36.435992


In [8]:
sales_file_2019Q3=list(recursive_file_gen("/home/jian/BigLots/2019_by_weeks/"))
sales_file_2019Q3=[x for x in sales_file_2019Q3 if x.split("MediaStorm_")[1][:10]>="2019-08-05" and x.split("MediaStorm_")[1][:10]<="2019-09-14" and "Daily" in x]
len(sales_file_2019Q3)


6

In [9]:
# First 6 weeks only
sales_file_2018Q3=glob.glob("/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/*.txt")
sales_file_2018Q3=[x for x in sales_file_2018Q3 if x.split("MediaStormDailySalesHistory")[1][:8]>"20180804" and x.split("MediaStormDailySalesHistory")[1][:8]<="20180915"]
len(sales_file_2018Q3)

6

In [11]:
df_agg_POS_data_2018Q3_rewards=pd.DataFrame()

i_counter=0
for file in sales_file_2018Q3:
    df=pd.read_table(file,dtype=str,sep="|",usecols=['location_id','transaction_dt','transaction_id','customer_id_hashed','class_code_id',
                                                     'item_transaction_units','item_transaction_amt'])
    df=df[pd.notnull(df['customer_id_hashed'])]
    df['item_transaction_units']=df['item_transaction_units'].astype(int)
    df['item_transaction_amt']=df['item_transaction_amt'].astype(float)
    
    df_sales=df.groupby(['location_id','transaction_dt','customer_id_hashed','class_code_id'])['item_transaction_units','item_transaction_amt'].sum().reset_index()
    df_sales=df_sales.rename(columns={"item_transaction_units":"units","item_transaction_amt":"sales"})
    
    df_trans=df[['location_id','transaction_dt','transaction_id','customer_id_hashed','class_code_id']].drop_duplicates()
    df_trans['trans']=1
    df_trans=df_trans.groupby(['location_id','transaction_dt','customer_id_hashed','class_code_id'])['trans'].sum().to_frame().reset_index()
    
    df=pd.merge(df_sales,df_trans,on=["location_id",'transaction_dt','customer_id_hashed','class_code_id'],how="outer")
    df_agg_POS_data_2018Q3_rewards=df_agg_POS_data_2018Q3_rewards.append(df)
    
    print(datetime.datetime.now(),i_counter,df_agg_POS_data_2018Q3_rewards.shape)
    i_counter+=1
gc.collect()

2019-09-19 11:25:32.326172 0 (5073014, 7)
2019-09-19 11:29:41.244508 1 (10434174, 7)
2019-09-19 11:35:43.367113 2 (15684268, 7)
2019-09-19 11:41:27.599695 3 (20972585, 7)
2019-09-19 11:43:44.021951 4 (26239492, 7)
2019-09-19 11:45:52.907062 5 (31250037, 7)


112

In [12]:
df_agg_POS_data_2019Q3_rewards=pd.DataFrame()

i_counter=0
for file in sales_file_2019Q3:
    df=pd.read_table(file,dtype=str,sep="|",usecols=['location_id','transaction_dt','transaction_id','customer_id_hashed','class_code_id',
                                                     'item_transaction_units','item_transaction_amt'])
    df=df[pd.notnull(df['customer_id_hashed'])]
    df['item_transaction_units']=df['item_transaction_units'].astype(int)
    df['item_transaction_amt']=df['item_transaction_amt'].astype(float)
    
    df_sales=df.groupby(['location_id','transaction_dt','customer_id_hashed','class_code_id'])['item_transaction_units','item_transaction_amt'].sum().reset_index()
    df_sales=df_sales.rename(columns={"item_transaction_units":"units","item_transaction_amt":"sales"})
    
    df_trans=df[['location_id','transaction_dt','transaction_id','customer_id_hashed','class_code_id']].drop_duplicates()
    df_trans['trans']=1
    df_trans=df_trans.groupby(['location_id','transaction_dt','customer_id_hashed','class_code_id'])['trans'].sum().to_frame().reset_index()
    
    df=pd.merge(df_sales,df_trans,on=["location_id",'transaction_dt','customer_id_hashed','class_code_id'],how="outer")
    df_agg_POS_data_2019Q3_rewards=df_agg_POS_data_2019Q3_rewards.append(df)
    
    print(datetime.datetime.now(),i_counter,df_agg_POS_data_2019Q3_rewards.shape)
    i_counter+=1
gc.collect()

2019-09-19 11:47:06.636305 0 (5384758, 7)
2019-09-19 11:48:27.058804 1 (10980271, 7)
2019-09-19 11:50:24.984047 2 (16585638, 7)
2019-09-19 11:52:20.871793 3 (22411370, 7)
2019-09-19 11:53:59.604035 4 (28046903, 7)
2019-09-19 11:55:56.083952 5 (33417444, 7)


112

In [13]:
def week_end_dt(x):
    y=datetime.datetime.strptime(x,"%Y-%m-%d").date()
    if y.weekday()==6:
        y=y+datetime.timedelta(days=6)
    else:
        y=y+datetime.timedelta(days=5-y.weekday())
    return y

In [14]:
df_agg_POS_data_2019Q3_rewards['Store_Type']=np.where(df_agg_POS_data_2019Q3_rewards['location_id']=="6990","Online",
                                                      np.where(df_agg_POS_data_2019Q3_rewards['location_id'].isin(set_SOTF_number),"SOTF","Other")
                                                     )
df_agg_POS_data_2018Q3_rewards['Store_Type']=np.where(df_agg_POS_data_2018Q3_rewards['location_id']=="6990","Online",
                                                      np.where(df_agg_POS_data_2018Q3_rewards['location_id'].isin(set_SOTF_number),"SOTF","Other")
                                                     )

In [15]:
df_week_end_dt=df_agg_POS_data_2019Q3_rewards[['transaction_dt']].drop_duplicates()
df_week_end_dt_2=df_agg_POS_data_2018Q3_rewards[['transaction_dt']].drop_duplicates()
df_week_end_dt=df_week_end_dt.append(df_week_end_dt_2)
del df_week_end_dt_2
df_week_end_dt['week_end_dt']=df_week_end_dt['transaction_dt'].apply(week_end_dt)


In [16]:
df_agg_POS_data_2019Q3_rewards=pd.merge(df_agg_POS_data_2019Q3_rewards,df_week_end_dt,on="transaction_dt",how="left")
df_agg_POS_data_2018Q3_rewards=pd.merge(df_agg_POS_data_2018Q3_rewards,df_week_end_dt,on="transaction_dt",how="left")

In [11]:
# Continue here
print(datetime.datetime.now())

2019-09-19 12:36:36.440652


In [8]:
write_folder="/home/jian/Projects/Big_Lots/Analysis/2019_Q3/Projection_est_in_Q3/new_audience_view/output_"+str(datetime.datetime.now().date())+"/"
try:
    os.stat(write_folder)
except:
    os.mkdir(write_folder)

In [18]:
'''
df_agg_POS_data_2019Q3_rewards.to_csv(write_folder+"df_agg_POS_data_2019Q3_rewards.csv",index=False)
df_agg_POS_data_2018Q3_rewards.to_csv(write_folder+"df_agg_POS_data_2018Q3_rewards.csv",index=False)
'''

In [12]:
df_agg_POS_data_2018Q3_rewards=pd.read_csv(write_folder+"df_agg_POS_data_2018Q3_rewards.csv",dtype=str)
df_agg_POS_data_2019Q3_rewards=pd.read_csv(write_folder+"df_agg_POS_data_2019Q3_rewards.csv",dtype=str)

df_agg_POS_data_2018Q3_rewards['units']=df_agg_POS_data_2018Q3_rewards['units'].astype(int)
df_agg_POS_data_2018Q3_rewards['sales']=df_agg_POS_data_2018Q3_rewards['sales'].astype(float)
df_agg_POS_data_2018Q3_rewards['trans']=df_agg_POS_data_2018Q3_rewards['trans'].astype(int)

df_agg_POS_data_2019Q3_rewards['units']=df_agg_POS_data_2019Q3_rewards['units'].astype(int)
df_agg_POS_data_2019Q3_rewards['sales']=df_agg_POS_data_2019Q3_rewards['sales'].astype(float)
df_agg_POS_data_2019Q3_rewards['trans']=df_agg_POS_data_2019Q3_rewards['trans'].astype(int)


In [13]:
df_2018Q3_by_id=df_agg_POS_data_2018Q3_rewards.groupby(['customer_id_hashed','Store_Type','class_code_id'])['sales','units','trans'].sum().reset_index()
df_2019Q3_by_id=df_agg_POS_data_2019Q3_rewards.groupby(['customer_id_hashed','Store_Type','class_code_id'])['sales','units','trans'].sum().reset_index()

# df_2018Q3_by_id=df_2018Q3_by_id.pivot_table(index=['customer_id_hashed'],columns=['Store_Type'],values=['sales','units','trans'])
# df_2019Q3_by_id=df_2019Q3_by_id.pivot_table(index=['customer_id_hashed'],columns=['Store_Type'],values=['sales','units','trans'])

# df_2018Q3_by_id=df_2018Q3_by_id.fillna(0).reset_index()
# df_2019Q3_by_id=df_2019Q3_by_id.fillna(0).reset_index()

In [14]:
df_2019Q3_by_id=pd.merge(df_2019Q3_by_id,df_audience_2019Q3,on="customer_id_hashed",how="left")
df_2018Q3_by_id=pd.merge(df_2018Q3_by_id,df_audience_2019Q3,on="customer_id_hashed",how="left")

In [15]:
df_2019Q3_by_id['year']=2019
df_2018Q3_by_id['year']=2018

In [16]:
df_18_shopper=df_2018Q3_by_id[['customer_id_hashed','year']].drop_duplicates()
df_19_shopper=df_2019Q3_by_id[['customer_id_hashed','year']].drop_duplicates()

df_shopper_type=df_18_shopper.append(df_19_shopper)
df_shopper_type=df_shopper_type.groupby(['customer_id_hashed'])['year'].sum().to_frame().reset_index()
df_shopper_type['shopper_type']=np.where(df_shopper_type['year']==2019,"Shopper_19Q3_Only",
                                        np.where(df_shopper_type['year']==2018,"Shopper_18Q3_Only","Shopper_Both")
                                        )

In [17]:
del df_shopper_type['year']
del df_18_shopper
del df_19_shopper
df_shopper_type.head(2)

,customer_id_hashed,shopper_type
0,000000ebcf6c6a2f4302291cc9babb0760208fc683b3b5...,Shopper_19Q3_Only
1,000006ff76d207f0c8fbb0262fb87c64b31a75dc89fd07...,Shopper_Both


In [18]:
df_2019Q3_by_id=pd.merge(df_2019Q3_by_id,df_shopper_type,on="customer_id_hashed",how="left")
df_2018Q3_by_id=pd.merge(df_2018Q3_by_id,df_shopper_type,on="customer_id_hashed",how="left")

In [19]:
df_2019Q3_by_id['register_group']=np.where(df_2019Q3_by_id['customer_id_hashed'].isin(set_new_signed_ids),"Registered_After_18Q2",
                                           "Registered_Before_18Q3")
df_2018Q3_by_id['register_group']=np.where(df_2018Q3_by_id['customer_id_hashed'].isin(set_new_signed_ids),"Registered_After_18Q2",
                                           "Registered_Before_18Q3")


In [20]:
df_2018Q3_by_id.head(2)

,customer_id_hashed,Store_Type,class_code_id,sales,units,trans,segment,year,shopper_type,register_group
0,000006ff76d207f0c8fbb0262fb87c64b31a75dc89fd07...,Other,11003,3.0,1,1,T_P_L_2019Q3_RFM,2018,Shopper_Both,Registered_After_18Q2
1,000006ff76d207f0c8fbb0262fb87c64b31a75dc89fd07...,Other,11101,1.9,1,1,T_P_L_2019Q3_RFM,2018,Shopper_Both,Registered_After_18Q2


In [21]:
df_2019Q3_by_id.to_csv(write_folder+"df_2019Q3_by_id.csv",index=False)
df_2018Q3_by_id.to_csv(write_folder+"df_2018Q3_by_id.csv",index=False)

In [33]:
Product_Taxonomy=pd.read_table("/home/jian/BigLots/static_files/ProductTaxonomy/MediaStormProductTaxonomy20190901-135942-034.txt",
                              sep="|",dtype=str)
Product_Taxonomy=Product_Taxonomy[['class_code_id','department_id','division_id']].drop_duplicates()
Product_Taxonomy.shape,Product_Taxonomy['class_code_id'].nunique()

((462, 3), 462)

In [34]:
Product_Taxonomy.head(2)

,class_code_id,department_id,division_id
0,11001,108,1
7,11002,108,1


In [36]:
Product_Taxonomy_name=Product_Taxonomy.copy()
division_name=pd.read_table("/home/jian/BigLots/static_files/MediaStorm Data Extract - Division Names.txt",sep="|",dtype=str)
department_name=pd.read_table("/home/jian/BigLots/static_files/MediaStorm Data Extract - Department Names.txt",sep="|",dtype=str)
class_name=pd.read_table("/home/jian/BigLots/static_files/MediaStorm Data Extract - Class Names.txt",sep="|",dtype=str,encoding="iso-8859-1")
Product_Taxonomy_name=pd.merge(Product_Taxonomy_name,division_name,on="division_id",how="left")
Product_Taxonomy_name=pd.merge(Product_Taxonomy_name,department_name,on="department_id",how="left")
Product_Taxonomy_name=pd.merge(Product_Taxonomy_name,class_name,on="class_code_id",how="left")
Product_Taxonomy_name.shape

(462, 6)

In [40]:
df_2018Q3_by_id=pd.merge(df_2018Q3_by_id,Product_Taxonomy,on="class_code_id",how="left")
df_2019Q3_by_id=pd.merge(df_2019Q3_by_id,Product_Taxonomy,on="class_code_id",how="left")
# All matched

In [79]:
Product_Taxonomy_name=Product_Taxonomy_name[['division_id','department_id','class_code_id','division_desc','department_desc','class_code_desc']]

In [80]:
Product_Taxonomy_name.head(2)

,division_id,department_id,class_code_id,division_desc,department_desc,class_code_desc
0,1,108,11001,FOOD,CANDYSNACK,CANDY
1,1,108,11002,FOOD,CANDYSNACK,SEASONAL CANDY


In [ ]:
# Rename the register group
df_2018Q3_by_id['register_group']=df_2018Q3_by_id['register_group'].replace("Registered_After_18Q2","Registered_Since_18Q3")
df_2019Q3_by_id['register_group']=df_2019Q3_by_id['register_group'].replace("Registered_After_18Q2","Registered_Since_18Q3")


# Class_Level

In [111]:
# The intermediate df is in the same and overwrote accross different product views

agg_func={"sales":"sum","units":"sum","trans":"sum","customer_id_hashed":"nunique"}
df_2019Q3_by_group_storetype=df_2019Q3_by_id.groupby(['segment','shopper_type','register_group','Store_Type','division_id','department_id','class_code_id'])['sales','units','trans','customer_id_hashed'].agg(agg_func).reset_index()
df_2018Q3_by_group_storetype=df_2018Q3_by_id.groupby(['segment','shopper_type','register_group','Store_Type','division_id','department_id','class_code_id'])['sales','units','trans','customer_id_hashed'].agg(agg_func).reset_index()

df_2019Q3_by_group_NoStoretype=df_2019Q3_by_id.groupby(['segment','shopper_type','register_group','division_id','department_id','class_code_id'])['sales','units','trans','customer_id_hashed'].agg(agg_func).reset_index()
df_2018Q3_by_group_NoStoretype=df_2018Q3_by_id.groupby(['segment','shopper_type','register_group','division_id','department_id','class_code_id'])['sales','units','trans','customer_id_hashed'].agg(agg_func).reset_index()


In [112]:
df_2019Q3_by_group_storetype['year']=2019
df_2018Q3_by_group_storetype['year']=2018

df_2019Q3_by_group_NoStoretype['year']=2019
df_2018Q3_by_group_NoStoretype['year']=2018

df_2019Q3_by_group_storetype=df_2019Q3_by_group_storetype.rename(columns={"customer_id_hashed":"shopper_id_count"})
df_2018Q3_by_group_storetype=df_2018Q3_by_group_storetype.rename(columns={"customer_id_hashed":"shopper_id_count"})
df_2019Q3_by_group_NoStoretype=df_2019Q3_by_group_NoStoretype.rename(columns={"customer_id_hashed":"shopper_id_count"})
df_2018Q3_by_group_NoStoretype=df_2018Q3_by_group_NoStoretype.rename(columns={"customer_id_hashed":"shopper_id_count"})


In [113]:
df_by_gorup_storetype=df_2019Q3_by_group_storetype.append(df_2018Q3_by_group_storetype)
df_by_gorup_NoStoretype=df_2019Q3_by_group_NoStoretype.append(df_2018Q3_by_group_NoStoretype)

In [114]:
df_by_gorup_storetype_wide_2019=df_2019Q3_by_group_storetype.pivot_table(index=["segment","shopper_type","register_group",'division_id','department_id','class_code_id'],columns="Store_Type",values=["sales","units","trans","shopper_id_count"]).reset_index()
df_by_gorup_storetype_wide_2018=df_2018Q3_by_group_storetype.pivot_table(index=["segment","shopper_type","register_group",'division_id','department_id','class_code_id'],columns="Store_Type",values=["sales","units","trans","shopper_id_count"]).reset_index()


In [115]:
new_col_list=[]
for col in df_by_gorup_storetype_wide_2019.columns.tolist():
    new_col="_".join(col)
    if new_col[-1]=="_":
        new_col=new_col[:-1]
    new_col_list.append(new_col)
df_by_gorup_storetype_wide_2019.columns=new_col_list

new_col_list=[]
for col in df_by_gorup_storetype_wide_2018.columns.tolist():
    new_col="_".join(col)
    if new_col[-1]=="_":
        new_col=new_col[:-1]
    new_col_list.append(new_col)
df_by_gorup_storetype_wide_2018.columns=new_col_list

In [116]:
df_by_gorup_storetype_wide_2019=df_by_gorup_storetype_wide_2019.fillna(0)
df_by_gorup_storetype_wide_2018=df_by_gorup_storetype_wide_2018.fillna(0)

In [117]:
count_aud_reg=df_audience_2019Q3.copy()

count_aud_reg['register_group']=np.where(count_aud_reg['customer_id_hashed'].isin(set_new_signed_ids),"Registered_After_18Q2",
                                           "Registered_Before_18Q3")
count_aud_reg=count_aud_reg.groupby(['segment','register_group'])['customer_id_hashed'].nunique().to_frame().reset_index()

In [118]:
df_unique_product=Product_Taxonomy_name[['division_id','department_id','class_code_id','division_desc','department_desc','class_code_desc']].drop_duplicates()
df_unique_segment=df_by_gorup_storetype[['segment']].drop_duplicates()
df_unique_shopper_type=df_by_gorup_storetype[['shopper_type']].drop_duplicates()
df_unique_register_group=df_by_gorup_storetype[['register_group']].drop_duplicates()

df_unique_product['key']=1
df_unique_segment['key']=1
df_unique_shopper_type['key']=1
df_unique_register_group['key']=1

df_unique_combination=pd.merge(df_unique_product,df_unique_segment,on="key",how="outer")
df_unique_combination=pd.merge(df_unique_combination,df_unique_shopper_type,on="key",how="outer")
df_unique_combination=pd.merge(df_unique_combination,df_unique_register_group,on="key",how="outer")

del df_unique_combination['key']
gc.collect()

df_unique_combination.shape

(19404, 9)

In [119]:
df_unique_combination['division_id']=df_unique_combination['division_id'].astype(int)
df_unique_combination['department_id']=df_unique_combination['department_id'].astype(int)
df_unique_combination['class_code_id']=df_unique_combination['class_code_id'].astype(int)
df_unique_combination=df_unique_combination.sort_values(['division_id','department_id','class_code_id','segment','shopper_type','register_group'])

df_by_gorup_storetype['division_id']=df_by_gorup_storetype['division_id'].astype(int)
df_by_gorup_storetype['department_id']=df_by_gorup_storetype['department_id'].astype(int)
df_by_gorup_storetype['class_code_id']=df_by_gorup_storetype['class_code_id'].astype(int)

df_by_gorup_NoStoretype['division_id']=df_by_gorup_NoStoretype['division_id'].astype(int)
df_by_gorup_NoStoretype['department_id']=df_by_gorup_NoStoretype['department_id'].astype(int)
df_by_gorup_NoStoretype['class_code_id']=df_by_gorup_NoStoretype['class_code_id'].astype(int)


In [120]:
writer=pd.ExcelWriter(write_folder+"BL_table_view_by_audience_ProductClass_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
df_unique_combination.to_excel(writer,"table",index=False)
df_by_gorup_storetype.to_excel(writer,"df_by_gorup_storetype",index=False)
df_by_gorup_NoStoretype.to_excel(writer,"by_gorup_NoStoretype",index=False)
df_by_gorup_storetype_wide_2019.to_excel(writer,"by_gorup_storetype_19_wide",index=False)
df_by_gorup_storetype_wide_2018.to_excel(writer,"by_gorup_storetype_18_wide",index=False)
df_audience_total_count_original.to_excel(writer,"audience_original_count",index=False)
df_audience_total_count.to_excel(writer,"audience_PS_HML_count",index=False)
count_aud_reg.to_excel(writer,"audience_Register_count",index=False)
Product_Taxonomy_name.to_excel(writer,"product_names",index=False)
writer.save()

# Department Level

In [121]:
# The intermediate df is in the same and overwrote accross different product views

agg_func={"sales":"sum","units":"sum","trans":"sum","customer_id_hashed":"nunique"}
df_2019Q3_by_group_storetype=df_2019Q3_by_id.groupby(['segment','shopper_type','register_group','Store_Type','division_id','department_id'])['sales','units','trans','customer_id_hashed'].agg(agg_func).reset_index()
df_2018Q3_by_group_storetype=df_2018Q3_by_id.groupby(['segment','shopper_type','register_group','Store_Type','division_id','department_id'])['sales','units','trans','customer_id_hashed'].agg(agg_func).reset_index()

df_2019Q3_by_group_NoStoretype=df_2019Q3_by_id.groupby(['segment','shopper_type','register_group','division_id','department_id'])['sales','units','trans','customer_id_hashed'].agg(agg_func).reset_index()
df_2018Q3_by_group_NoStoretype=df_2018Q3_by_id.groupby(['segment','shopper_type','register_group','division_id','department_id'])['sales','units','trans','customer_id_hashed'].agg(agg_func).reset_index()


In [122]:
df_2019Q3_by_group_storetype['year']=2019
df_2018Q3_by_group_storetype['year']=2018

df_2019Q3_by_group_NoStoretype['year']=2019
df_2018Q3_by_group_NoStoretype['year']=2018

df_2019Q3_by_group_storetype=df_2019Q3_by_group_storetype.rename(columns={"customer_id_hashed":"shopper_id_count"})
df_2018Q3_by_group_storetype=df_2018Q3_by_group_storetype.rename(columns={"customer_id_hashed":"shopper_id_count"})
df_2019Q3_by_group_NoStoretype=df_2019Q3_by_group_NoStoretype.rename(columns={"customer_id_hashed":"shopper_id_count"})
df_2018Q3_by_group_NoStoretype=df_2018Q3_by_group_NoStoretype.rename(columns={"customer_id_hashed":"shopper_id_count"})


In [123]:
df_by_gorup_storetype=df_2019Q3_by_group_storetype.append(df_2018Q3_by_group_storetype)
df_by_gorup_NoStoretype=df_2019Q3_by_group_NoStoretype.append(df_2018Q3_by_group_NoStoretype)

In [124]:
df_by_gorup_storetype_wide_2019=df_2019Q3_by_group_storetype.pivot_table(index=["segment","shopper_type","register_group",'division_id','department_id'],columns="Store_Type",values=["sales","units","trans","shopper_id_count"]).reset_index()
df_by_gorup_storetype_wide_2018=df_2018Q3_by_group_storetype.pivot_table(index=["segment","shopper_type","register_group",'division_id','department_id'],columns="Store_Type",values=["sales","units","trans","shopper_id_count"]).reset_index()


In [125]:
new_col_list=[]
for col in df_by_gorup_storetype_wide_2019.columns.tolist():
    new_col="_".join(col)
    if new_col[-1]=="_":
        new_col=new_col[:-1]
    new_col_list.append(new_col)
df_by_gorup_storetype_wide_2019.columns=new_col_list

new_col_list=[]
for col in df_by_gorup_storetype_wide_2018.columns.tolist():
    new_col="_".join(col)
    if new_col[-1]=="_":
        new_col=new_col[:-1]
    new_col_list.append(new_col)
df_by_gorup_storetype_wide_2018.columns=new_col_list

In [126]:
df_by_gorup_storetype_wide_2019=df_by_gorup_storetype_wide_2019.fillna(0)
df_by_gorup_storetype_wide_2018=df_by_gorup_storetype_wide_2018.fillna(0)

In [127]:
df_unique_product=Product_Taxonomy_name[['division_id','department_id','division_desc','department_desc']].drop_duplicates()
df_unique_segment=df_by_gorup_storetype[['segment']].drop_duplicates()
df_unique_shopper_type=df_by_gorup_storetype[['shopper_type']].drop_duplicates()
df_unique_register_group=df_by_gorup_storetype[['register_group']].drop_duplicates()

df_unique_product['key']=1
df_unique_segment['key']=1
df_unique_shopper_type['key']=1
df_unique_register_group['key']=1

df_unique_combination=pd.merge(df_unique_product,df_unique_segment,on="key",how="outer")
df_unique_combination=pd.merge(df_unique_combination,df_unique_shopper_type,on="key",how="outer")
df_unique_combination=pd.merge(df_unique_combination,df_unique_register_group,on="key",how="outer")

del df_unique_combination['key']
gc.collect()

df_unique_combination.shape

(2520, 7)

In [128]:
df_unique_combination['division_id']=df_unique_combination['division_id'].astype(int)
df_unique_combination['department_id']=df_unique_combination['department_id'].astype(int)
# df_unique_combination['class_code_id']=df_unique_combination['class_code_id'].astype(int)
df_unique_combination=df_unique_combination.sort_values(['division_id','department_id','segment','shopper_type','register_group'])

df_by_gorup_storetype['division_id']=df_by_gorup_storetype['division_id'].astype(int)
df_by_gorup_storetype['department_id']=df_by_gorup_storetype['department_id'].astype(int)
# df_by_gorup_storetype['class_code_id']=df_by_gorup_storetype['class_code_id'].astype(int)

df_by_gorup_NoStoretype['division_id']=df_by_gorup_NoStoretype['division_id'].astype(int)
df_by_gorup_NoStoretype['department_id']=df_by_gorup_NoStoretype['department_id'].astype(int)
# df_by_gorup_NoStoretype['class_code_id']=df_by_gorup_NoStoretype['class_code_id'].astype(int)


In [129]:
writer=pd.ExcelWriter(write_folder+"BL_table_view_by_audience_ProductDepartment_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
df_unique_combination.to_excel(writer,"table",index=False)
df_by_gorup_storetype.to_excel(writer,"df_by_gorup_storetype",index=False)
df_by_gorup_NoStoretype.to_excel(writer,"by_gorup_NoStoretype",index=False)
df_by_gorup_storetype_wide_2019.to_excel(writer,"by_gorup_storetype_19_wide",index=False)
df_by_gorup_storetype_wide_2018.to_excel(writer,"by_gorup_storetype_18_wide",index=False)
df_audience_total_count_original.to_excel(writer,"audience_original_count",index=False)
df_audience_total_count.to_excel(writer,"audience_PS_HML_count",index=False)
count_aud_reg.to_excel(writer,"audience_Register_count",index=False)
Product_Taxonomy_name.to_excel(writer,"product_names",index=False)
writer.save()

# Division_Level

In [130]:
# The intermediate df is in the same and overwrote accross different product views

agg_func={"sales":"sum","units":"sum","trans":"sum","customer_id_hashed":"nunique"}
df_2019Q3_by_group_storetype=df_2019Q3_by_id.groupby(['segment','shopper_type','register_group','Store_Type','division_id'])['sales','units','trans','customer_id_hashed'].agg(agg_func).reset_index()
df_2018Q3_by_group_storetype=df_2018Q3_by_id.groupby(['segment','shopper_type','register_group','Store_Type','division_id'])['sales','units','trans','customer_id_hashed'].agg(agg_func).reset_index()

df_2019Q3_by_group_NoStoretype=df_2019Q3_by_id.groupby(['segment','shopper_type','register_group','division_id'])['sales','units','trans','customer_id_hashed'].agg(agg_func).reset_index()
df_2018Q3_by_group_NoStoretype=df_2018Q3_by_id.groupby(['segment','shopper_type','register_group','division_id'])['sales','units','trans','customer_id_hashed'].agg(agg_func).reset_index()


In [131]:
df_2019Q3_by_group_storetype['year']=2019
df_2018Q3_by_group_storetype['year']=2018

df_2019Q3_by_group_NoStoretype['year']=2019
df_2018Q3_by_group_NoStoretype['year']=2018

df_2019Q3_by_group_storetype=df_2019Q3_by_group_storetype.rename(columns={"customer_id_hashed":"shopper_id_count"})
df_2018Q3_by_group_storetype=df_2018Q3_by_group_storetype.rename(columns={"customer_id_hashed":"shopper_id_count"})
df_2019Q3_by_group_NoStoretype=df_2019Q3_by_group_NoStoretype.rename(columns={"customer_id_hashed":"shopper_id_count"})
df_2018Q3_by_group_NoStoretype=df_2018Q3_by_group_NoStoretype.rename(columns={"customer_id_hashed":"shopper_id_count"})


In [132]:
df_by_gorup_storetype=df_2019Q3_by_group_storetype.append(df_2018Q3_by_group_storetype)
df_by_gorup_NoStoretype=df_2019Q3_by_group_NoStoretype.append(df_2018Q3_by_group_NoStoretype)

In [133]:
df_by_gorup_storetype_wide_2019=df_2019Q3_by_group_storetype.pivot_table(index=["segment","shopper_type","register_group",'division_id'],columns="Store_Type",values=["sales","units","trans","shopper_id_count"]).reset_index()
df_by_gorup_storetype_wide_2018=df_2018Q3_by_group_storetype.pivot_table(index=["segment","shopper_type","register_group",'division_id'],columns="Store_Type",values=["sales","units","trans","shopper_id_count"]).reset_index()


In [134]:
new_col_list=[]
for col in df_by_gorup_storetype_wide_2019.columns.tolist():
    new_col="_".join(col)
    if new_col[-1]=="_":
        new_col=new_col[:-1]
    new_col_list.append(new_col)
df_by_gorup_storetype_wide_2019.columns=new_col_list

new_col_list=[]
for col in df_by_gorup_storetype_wide_2018.columns.tolist():
    new_col="_".join(col)
    if new_col[-1]=="_":
        new_col=new_col[:-1]
    new_col_list.append(new_col)
df_by_gorup_storetype_wide_2018.columns=new_col_list

In [135]:
df_by_gorup_storetype_wide_2019=df_by_gorup_storetype_wide_2019.fillna(0)
df_by_gorup_storetype_wide_2018=df_by_gorup_storetype_wide_2018.fillna(0)

In [136]:
df_unique_product=Product_Taxonomy_name[['division_id','division_desc']].drop_duplicates()
df_unique_segment=df_by_gorup_storetype[['segment']].drop_duplicates()
df_unique_shopper_type=df_by_gorup_storetype[['shopper_type']].drop_duplicates()
df_unique_register_group=df_by_gorup_storetype[['register_group']].drop_duplicates()

df_unique_product['key']=1
df_unique_segment['key']=1
df_unique_shopper_type['key']=1
df_unique_register_group['key']=1

df_unique_combination=pd.merge(df_unique_product,df_unique_segment,on="key",how="outer")
df_unique_combination=pd.merge(df_unique_combination,df_unique_shopper_type,on="key",how="outer")
df_unique_combination=pd.merge(df_unique_combination,df_unique_register_group,on="key",how="outer")

del df_unique_combination['key']
gc.collect()

df_unique_combination.shape

(378, 5)

In [137]:
df_unique_combination['division_id']=df_unique_combination['division_id'].astype(int)
# df_unique_combination['department_id']=df_unique_combination['department_id'].astype(int)
# df_unique_combination['class_code_id']=df_unique_combination['class_code_id'].astype(int)
df_unique_combination=df_unique_combination.sort_values(['division_id','segment','shopper_type','register_group'])

df_by_gorup_storetype['division_id']=df_by_gorup_storetype['division_id'].astype(int)
# df_by_gorup_storetype['department_id']=df_by_gorup_storetype['department_id'].astype(int)
# df_by_gorup_storetype['class_code_id']=df_by_gorup_storetype['class_code_id'].astype(int)

df_by_gorup_NoStoretype['division_id']=df_by_gorup_NoStoretype['division_id'].astype(int)
# df_by_gorup_NoStoretype['department_id']=df_by_gorup_NoStoretype['department_id'].astype(int)
# df_by_gorup_NoStoretype['class_code_id']=df_by_gorup_NoStoretype['class_code_id'].astype(int)


In [138]:
writer=pd.ExcelWriter(write_folder+"BL_table_view_by_audience_ProductDivision_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
df_unique_combination.to_excel(writer,"table",index=False)
df_by_gorup_storetype.to_excel(writer,"df_by_gorup_storetype",index=False)
df_by_gorup_NoStoretype.to_excel(writer,"by_gorup_NoStoretype",index=False)
df_by_gorup_storetype_wide_2019.to_excel(writer,"by_gorup_storetype_19_wide",index=False)
df_by_gorup_storetype_wide_2018.to_excel(writer,"by_gorup_storetype_18_wide",index=False)
df_audience_total_count_original.to_excel(writer,"audience_original_count",index=False)
df_audience_total_count.to_excel(writer,"audience_PS_HML_count",index=False)
count_aud_reg.to_excel(writer,"audience_Register_count",index=False)
Product_Taxonomy_name.to_excel(writer,"product_names",index=False)
writer.save()